In [ ]:
import cv2
import json
import numpy as np
import skimage.draw
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa

In [ ]:
# Set output area size for image
plt.rcParams['figure.figsize'] = (15.0, 15.0)

## Load Data

In [ ]:
img = cv2.imread("test.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
height, width = img.shape[:2]
print(width, height)
plt.imshow(img)
plt.show()

## Check Various Augument Image

In [ ]:
aug = iaa.Affine(rotate=(-180, 180))
img_aug = aug.augment_image(img)
plt.imshow(img_aug)
plt.show()

In [ ]:
seq = iaa.Sequential([
#     iaa.CropToFixedSize(width=1080, height=1080, position="center"),
    iaa.Sometimes(0.8, iaa.Affine(rotate=(-180, 180))),
    iaa.LogContrast(gain=(0.6, 1.4)),
#     iaa.Sometimes(0.5, iaa.MultiplyHueAndSaturation(mul_saturation=(0.5, 1.5))),
#     iaa.Sometimes(0.5, iaa.Affine(shear=(-3, 3))),
#     iaa.Sometimes(0.5, iaa.PiecewiseAffine(scale=(0.01, 0.02))),
    iaa.GaussianBlur(sigma=(2.0, 3.0))
])

img_aug = seq.augment_image(img)
plt.imshow(img_aug)
plt.show()

## Load annotation file

In [ ]:
annotations = json.load(open("test_label.json"))
annotations = list(annotations.values())
annotations = [a for a in annotations if a['regions']]

polygons = []
names = []
for a in annotations:
    polygons = [r['shape_attributes'] for r in a['regions']] 
    names = [r['region_attributes'] for r in a['regions']]

## Check masked data

In [ ]:
mask = np.zeros([height, width, len(polygons)], dtype=np.uint8)
plt.imshow(img)
plt.show()
for i, (p, n) in enumerate(zip(polygons, names)):
    rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
    mask[rr, cc, i] = 1
    
    print(n["name"])
    plt.imshow(mask[:, :, i], cmap="gray")
    plt.show()

## Augument image and mask

In [ ]:
# Integrate image numpy and mask numpy
img_and_mask = np.append(img, mask, axis=2)

# Check source image
plt.imshow(img_and_mask[:,:,:3])
plt.show()

# Augment
aug = iaa.Affine(rotate=(-180, 180))
test = aug.augment_image(img_and_mask)
test_img = test[:,:,:3]
test_mask = test[:,:,3:]

# Check augmented image
plt.imshow(test_img)
plt.show()

# Save augmented image
# cv2.imwrite("test_augment.png", cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR))

# Make augmented masked label data
result = {}
result["test"] = {"filename":"test_augment.png", "file_attributes":{}, "size": 0}
regions = []
for i in range(test_mask.shape[-1]):
    attr = {}
    region_attributes = {"name": "test"}
    attr["region_attributes"] = region_attributes
    shape_attributes = {"name":"polyline"}
    all_points_x = []
    all_points_y = []
    
    tmp = test_mask[:,:,i]
    ret, thresh = cv2.threshold(tmp, 0.5, 1.0, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for i in range(len(contours[0])):
        all_points_x.append(int(contours[0][i][0][0]))
        all_points_y.append(int(contours[0][i][0][1]))
    shape_attributes["all_points_x"] = all_points_x
    shape_attributes["all_points_y"] = all_points_y
    
    attr["shape_attributes"] = shape_attributes
    regions.append(attr)
    plt.imshow(tmp, cmap="gray")
    plt.show()
    
result["test"]["regions"] = regions

In [ ]:
# Save json
with open("label.json", 'w') as f:
    json.dump(result, f)